# Importing the Library

In [6]:
import os
import json
import pandas as pd
import traceback

In [152]:
from dotenv import load_dotenv
load_dotenv() #takes environment variable from .env

True

In [153]:
KEY=os.getenv("GOOGLE_API_KEY")

In [161]:
from langchain_google_genai  import ChatGoogleGenerativeAI
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2

In [202]:
llm = ChatGoogleGenerativeAI(
    google_api_key = KEY,
    model="gemini-pro",
    temperature=0.5
    )


In [203]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [204]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [205]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [206]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [207]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [208]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [209]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [210]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [211]:
import os
file_path = "e:\\Personal Files\\Learning Programming\\Data Science\\Krish\\Gen AI\\Practical\\MCQGen\\data.txt"

In [212]:
file_path

'e:\\Personal Files\\Learning Programming\\Data Science\\Krish\\Gen AI\\Practical\\MCQGen\\data.txt'

In [213]:
with open(file_path,"r") as f:
    TEXT = f.read()


In [214]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine lea

In [215]:
#Searalize the python dictionary to JSON format
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [216]:
NUMBER = 5
SUBJECT = "machine learining"
TONE = "simple"

In [217]:
response = generate_evaluate_chain({
    "text" : TEXT,
    "number" : NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoreti

In [219]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for descri

In [236]:
response.get("quiz")

'### RESPONSE_JSON\n{"1": {"mcq": "What is the primary focus of machine learning?", "options": {"a": "Developing algorithms that can learn from data", "b": "Creating artificial intelligence systems", "c": "Analyzing large datasets", "d": "Solving complex mathematical problems"}, "correct": "a"}, "2": {"mcq": "Which decade saw significant advancements in machine learning?", "options": {"a": "1950s", "b": "1960s", "c": "1970s", "d": "2010s"}, "correct": "d"}, "3": {"mcq": "What is a key application of machine learning in business?", "options": {"a": "Predictive analytics", "b": "Customer relationship management", "c": "Data visualization", "d": "Financial modeling"}, "correct": "a"}, "4": {"mcq": "Who is considered a pioneer in the field of machine learning?", "options": {"a": "Arthur Samuel", "b": "Alan Turing", "c": "Donald Hebb", "d": "Raytheon Company"}, "correct": "a"}, "5": {"mcq": "What is the purpose of supervised learning in machine learning?", "options": {"a": "Classifying data

In [239]:
response.get("quiz").split("\n")

['### RESPONSE_JSON',
 '{"1": {"mcq": "What is the primary focus of machine learning?", "options": {"a": "Developing algorithms that can learn from data", "b": "Creating artificial intelligence systems", "c": "Analyzing large datasets", "d": "Solving complex mathematical problems"}, "correct": "a"}, "2": {"mcq": "Which decade saw significant advancements in machine learning?", "options": {"a": "1950s", "b": "1960s", "c": "1970s", "d": "2010s"}, "correct": "d"}, "3": {"mcq": "What is a key application of machine learning in business?", "options": {"a": "Predictive analytics", "b": "Customer relationship management", "c": "Data visualization", "d": "Financial modeling"}, "correct": "a"}, "4": {"mcq": "Who is considered a pioneer in the field of machine learning?", "options": {"a": "Arthur Samuel", "b": "Alan Turing", "c": "Donald Hebb", "d": "Raytheon Company"}, "correct": "a"}, "5": {"mcq": "What is the purpose of supervised learning in machine learning?", "options": {"a": "Classifying 

In [246]:
quiz = response.get("quiz").split("\n")[1]


str

In [247]:
quiz = json.loads(quiz)

In [251]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [253]:
quiz_table_data

[{'MCQ': 'What is the primary focus of machine learning?',
  'Choices': 'a: Developing algorithms that can learn from data | b: Creating artificial intelligence systems | c: Analyzing large datasets | d: Solving complex mathematical problems',
  'Correct': 'a'},
 {'MCQ': 'Which decade saw significant advancements in machine learning?',
  'Choices': 'a: 1950s | b: 1960s | c: 1970s | d: 2010s',
  'Correct': 'd'},
 {'MCQ': 'What is a key application of machine learning in business?',
  'Choices': 'a: Predictive analytics | b: Customer relationship management | c: Data visualization | d: Financial modeling',
  'Correct': 'a'},
 {'MCQ': 'Who is considered a pioneer in the field of machine learning?',
  'Choices': 'a: Arthur Samuel | b: Alan Turing | c: Donald Hebb | d: Raytheon Company',
  'Correct': 'a'},
 {'MCQ': 'What is the purpose of supervised learning in machine learning?',
  'Choices': 'a: Classifying data based on labeled examples | b: Finding patterns in unlabeled data | c: Making

In [255]:
quiz=pd.DataFrame(quiz_table_data)
quiz

,MCQ,Choices,Correct
0,What is the primary focus of machine learning?,a: Developing algorithms that can learn from d...,a
1,Which decade saw significant advancements in m...,a: 1950s | b: 1960s | c: 1970s | d: 2010s,d
2,What is a key application of machine learning ...,a: Predictive analytics | b: Customer relation...,a
3,Who is considered a pioneer in the field of ma...,a: Arthur Samuel | b: Alan Turing | c: Donald ...,a
4,What is the purpose of supervised learning in ...,a: Classifying data based on labeled examples ...,a


In [256]:
quiz.to_csv("machinelearning.csv",index=False)